In [1]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import importlib

DELETE_ALL = False
WRITE_DATA=False

# importlib.reload(db_info)

In [2]:
d1 = {
    1:'jan',
    2:'feb',
    3:'mar',
    4:'apr',
    5:'may',
    6:'jun',
    7:'jul',
    8:'aug',
    9:'sep',
    10:'oct',
    11:'nov',
    12:'dec'
}


In [11]:
zip_folder_parent = open('./temp_folder/zip_folder_parent.txt','r').read()
futures_parent = f'{zip_folder_parent}/futures'

all_names_ordered = []
for yy in np.arange(19,20):
    fnames = glob.glob(f'{futures_parent}/*{yy}.txt')
    d = {}
    for fname in fnames:
        mmm = fname.split('/')[-1].split('.txt')[0][0:-2][-3:]
        d[mmm] = fname
    fnames_ordered = [d[d1[m]] for m in d1.keys() if d1[m] in d]
    all_names_ordered += fnames_ordered
all_names_ordered

df_all = None
header = ['contract','month_year','yymmdd','open','high','low','close','volume','open_interest']

for fname in tqdm_notebook(all_names_ordered):
    df_temp = pd.read_csv(fname,header=None)
    df_temp.columns = header
    if df_all is None:
        df_all = df_temp.copy()
    else:
        df_all = df_all.append(df_temp,ignore_index=True)
        df_all.index = list(range(len(df_all)))
    

In [12]:
len(df_all)

250154

,contract,month_year,yymmdd,open,high,low,close,volume,open_interest
242995,JZ,2099Z,190625.0,0102061.4100093100093,0,0.0,NaN,0.0,NaN
247048,JZ,2099Z,190627.0,0101025100724100724,0,0.0,NaN,0.0,NaN


In [30]:
df_temp = df_all.copy()#.iloc[:1000]
isnas = df_temp.yymmdd.isna()
df_temp = df_temp[~isnas]
df_temp = df_temp[~df_temp.open_interest.isna()]
df_temp.volume = df_temp.volume.fillna(0)
df_temp = df_temp[df_temp.open.astype(str).str.count('\.')<=1]
df_temp.index = list(range(len(df_temp)))
df_temp.loc[df_temp.month_year=='Y','month_year'] = '2099Z'
symbols = df_temp.contract + df_temp.month_year.str.slice(-1,)  + df_temp.month_year.str.slice(2,4)
settle_dates = ('20' + df_temp.yymmdd.astype(str)).astype(float).astype(int)
opens = df_temp.open.astype(float)
highs = df_temp.high.astype(float)
lows = df_temp.low.astype(float)
closes = df_temp.close.astype(float)
volumes = df_temp.volume.astype(int)
open_interests = df_temp.open_interest.astype(int)
df_final = pd.DataFrame({'symbol':symbols,
    'settle_date':settle_dates,
    'open':opens,
    'high':highs,
    'low':lows,
    'close':closes,
    'adj_close':closes,
    'volume':volumes,
    'open_interest':open_interests})
df_final.head()

,symbol,settle_date,open,high,low,close,adj_close,volume,open_interest
0,NVZ99,20190101,36161.8000,36284.04000,35888.62000,36254.57000,36254.57000,0,0
1,A6Z99,20190102,0.7051,0.70519,0.69812,0.69833,0.69833,0,0
2,A6F19,20190102,0.7022,0.70440,0.69860,0.70000,0.70000,367,396
3,A6G19,20190102,0.7024,0.70530,0.69960,0.70050,0.70050,158,305
4,A6H19,20190102,0.7054,0.70590,0.69910,0.70080,0.70080,50795,120653


#### add month_num to df_final

In [31]:
df_monthnum = pd.read_csv('month_codes.csv')
df_monthnum

,month_num,mmm,month_code
0,1,jan,F
1,2,feb,G
2,3,mar,H
3,4,apr,J
4,5,may,K
5,6,jun,M
6,7,jul,N
7,8,aug,Q
8,9,sep,U
9,10,oct,V


In [32]:
df_final.symbol.str.slice(0,-3).unique()

array(['NV', 'A6', 'AE', 'AP', 'B6', 'BG', 'BT', 'CA', 'CB', 'CC', 'CE',
       'CG', 'CL', 'CT', 'D6', 'DL', 'DS', 'DX', 'DY', 'E6', 'EF', 'ES',
       'EW', 'F', 'FS', 'FX', 'G', 'GC', 'GD', 'GE', 'GF', 'GG', 'GH',
       'HE', 'HF', 'HG', 'HO', 'HR', 'HS', 'II', 'IM', 'IR', 'J6', 'JX',
       'JZ', 'KC', 'KE', 'KL', 'KS', 'L', 'L6', 'LE', 'LF', 'LS', 'LX',
       'M6', 'MJ', 'MW', 'MX', 'N6', 'NF', 'NG', 'NQ', 'NX', 'NY', 'OJ',
       'PA', 'PL', 'QR', 'RB', 'RM', 'RS', 'S6', 'SB', 'SD', 'SI', 'SL',
       'SP', 'SV', 'SW', 'T6', 'TI', 'TP', 'UD', 'VI', 'VS', 'WI', 'WT',
       'X', 'XG', 'XH', 'YC', 'YK', 'YM', 'YT', 'YU', 'YW', 'ZB', 'ZC',
       'ZF', 'ZK', 'ZL', 'ZM', 'ZN', 'ZO', 'ZQ', 'ZR', 'ZS', 'ZT', 'ZW',
       'CF', 'SZ'], dtype=object)

In [33]:
dfu2 = df_final.copy()
dfu2['contract'] = dfu2.symbol.str.slice(0,-3)
dfu2['year'] = dfu2.symbol.apply(lambda s: 2000 + int(s[-2:]))
dfu2['month_code'] = dfu2.symbol.str.slice(-3,-2)
dfu3 = dfu2.merge(df_monthnum,on='month_code',how='inner')
dfu3['yyyymm'] = dfu3.year*100+dfu3.month_num
display(dfu2.month_code.unique())
dfu4 = dfu3[['contract','symbol','settle_date','yyyymm']]
dfu4['contract_num'] =dfu4[['contract','settle_date','yyyymm']].groupby(['contract','settle_date']).yyyymm.rank()
dfu4['contract_num'] = dfu4['contract_num'].astype(int)
dfu4 = dfu4.sort_values(['settle_date','contract','yyyymm'])
dfu4.index = list(range(len(dfu4)))
print(len(df_final),len(dfu4))
dfu5 = df_final.merge(dfu4[['symbol','settle_date','contract_num']],on=['symbol','settle_date'])
dfu5.index = list(range(len(dfu5)))
dfu5.open=dfu5.open.round(8)
dfu5.high=dfu5.high.round(8)
dfu5.low=dfu5.low.round(8)
dfu5.close=dfu5.close.round(8)
dfu5.adj_close = dfu5.adj_close.round(8)



array(['Z', 'F', 'G', 'H', 'J', 'M', 'U', 'K', 'N', 'Q', 'V', 'X'],
      dtype=object)

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


250139 250139


In [34]:
len(df_all),len(df_final),len(dfu5)

(250154, 250139, 250139)

#### Are there dupes??

In [35]:
ag = ['symbol','settle_date']
df_counts = dfu5[ag+['close']].groupby(ag,as_index=False).count()
dupes_exist  = len(df_counts[df_counts.close>1])>0
dupes_exist

False

#### if there are dupes, get rid of them

In [36]:
if dupes_exist > 0:
    dfu5 = dfu5.drop_duplicates()
    dfu5.index = list(range(len(dfu5)))
    

#### show unique contract_num numbers

In [37]:
dfu5.contract_num.unique()

array([  1,  24,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  86,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85, 110,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 120, 111, 112, 113, 114, 115, 116,
       117, 118, 119])

### NOW WRITE THE DATA FOR ALL YEARs


#### First create an instance of PgPandas

In [38]:
csv_temp_path = './temp_folder/df_all_temp.csv'
print(os.path.abspath(csv_temp_path))
pga = db_info.get_db_info()
tbname = 'sec_schema.underlying_table'
print(tbname)
col_tuple_list =   [('symbol','text'),('settle_date','integer'),('contract_num','integer'),
     ('open','numeric'),('high','numeric'),('low','numeric'),('close','numeric'),
     ('adj_close','numeric'),('volume','integer'),('open_interest','integer')]
col_list = [l[0] for l in col_tuple_list]
print(col_list)
print(f'creating csv file {csv_temp_path}: {datetime.datetime.now()}')
dfu5[col_list].to_csv(csv_temp_path,index=False)


/Users/bperlman1/Documents/billybyte/pyliverisk/barchartacs/barchartacs/temp_folder/df_all_temp.csv
  sec_db
sec_schema.underlying_table
['symbol', 'settle_date', 'contract_num', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'open_interest']
creating csv file ./temp_folder/df_all_temp.csv: 2019-07-04 11:27:13.141618


/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### Delete all rows if necessary

In [43]:
DELETE_ALL,WRITE_DATA

(False, False)

In [41]:

if DELETE_ALL:
    # drop table
    sql = f"drop table IF EXISTS {tbname}"
    pga.exec_sql_raw(sql) 
    # re create it
    table_create = f'create table {tbname}('
    table_create += ','.join([f"{l[0]} {l[1]} not null" for l in col_tuple_list])
    table_create += ', primary key(symbol,settle_date));'
    pga.exec_sql_raw(table_create) 
    
pga.get_sql(f"select count(*) from {tbname}")

,count
0,4708633


In [42]:
col_list

['symbol',
 'settle_date',
 'contract_num',
 'open',
 'high',
 'low',
 'close',
 'adj_close',
 'volume',
 'open_interest']

In [44]:
if WRITE_DATA:
    clist = [l[0] for l in col_list]
    print(f'writing data all to database BEGIN: {datetime.datetime.now()}')
    abspath = os.path.abspath(csv_temp_path)
    sql = f"COPY {tbname} FROM '{abspath}' DELIMITER ',' CSV HEADER;"
#     sql = f"COPY sec_schema.underlying_table_test FROM '{abspath}' DELIMITER ',' CSV HEADER;"
    print(sql)
#     pga.exec_sql_raw(sql) 
    print('open Dbeaver and copy the sql COPY command into it and execute it')
    print(f'writing data all to database END: {datetime.datetime.now()}')
    pass

In [108]:
def get_data_by_contract(pga,contract,
            month_code_clause=None,
             settle_date_clause=None,
             limit=None):
    futtab = tbname
    cl_month_code = '' if month_code_clause is None else f"and substring(symbol,3,1) {month_code_clause}"
    cl_sd = '' if settle_date_clause is None else f"and 'settle_date {settle_date_clause}"
    clim = '' if limit is None else f"limit {limit}"
    other_criteria = f'{cl_month_code} {cl_sd} {clim}'
    sql = f"select * from {tbname} where substring(symbol,1,2) = '{contract}' {other_criteria};"
    print(sql)
    df=  pga.get_sql(sql)
    return df

dfc = get_data_by_contract(pga,'CB',limit=3)
display(dfc)
dfc = get_data_by_contract(pga,'CB',month_code_clause="in ('G','N')",limit=3)
display(dfc)


select * from sec_schema.underlying_table where substring(symbol,1,2) = 'CB'   limit 3;


,symbol,settle_date,contract_num,open,high,low,close,adj_close,volume,open_interest
0,CBZ99,20100901,70,75.64,75.64,75.64,75.64,75.64,0,0
1,CBV10,20100901,1,74.46,76.76,74.42,76.35,76.35,188333,208390
2,CBX10,20100901,2,74.76,77.14,74.75,76.72,76.72,90252,169853


select * from sec_schema.underlying_table where substring(symbol,1,2) = 'CB' and substring(symbol,3,1) in ('G','N')  limit 3;


,symbol,settle_date,contract_num,open,high,low,close,adj_close,volume,open_interest
0,CBG11,20100901,5,76.76,78.80,76.76,78.52,78.52,8166,26056
1,CBN11,20100901,10,0.00,81.24,81.24,81.24,81.24,1787,8237
2,CBG12,20100901,17,0.00,83.77,83.77,83.77,83.77,0,1579


In [109]:
pga.get_sql("select * from sec_schema.underlying_table limit 10")


,symbol,settle_date,contract_num,open,high,low,close,adj_close,volume,open_interest
0,A6Z99,20100901,7,0.8908,0.9115,0.8902,0.9115,0.9115,0,0
1,A6U10,20100901,1,0.8910,0.9104,0.8900,0.9072,0.9072,105813,98993
2,A6Z10,20100901,2,0.8820,0.9001,0.8808,0.8974,0.8974,451,3282
3,A6H11,20100901,3,0.0000,0.8878,0.8681,0.8878,0.8878,0,8
4,A6M11,20100901,4,0.0000,0.8782,0.8589,0.8782,0.8782,0,152
5,A6U11,20100901,5,0.0000,0.8686,0.8493,0.8686,0.8686,0,4
6,A6Z11,20100901,6,0.0000,0.8590,0.8397,0.8590,0.8590,0,0
7,ADZ99,20100901,7,0.8908,0.9115,0.8902,0.9115,0.9115,0,0
8,ADU10,20100901,1,0.9032,0.9080,0.9032,0.9072,0.9072,105813,98993
9,ADZ10,20100901,2,0.8972,0.8974,0.8972,0.8974,0.8974,451,3282
